In [1]:
!nvidia-smi

Fri Feb 17 11:13:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0%   32C    P8    25W / 460W |  20619MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1C:00.0 Off |                  N/A |
| 27%   

In [3]:
!pip install torch
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 31.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.8 MB/s eta 0:00:0000:0100:01


In [7]:
import numpy as np
import torch

In [8]:
torch.cuda.is_available()

True

In [9]:
torch.cuda.empty_cache()

# model

In [33]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

In [34]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [49]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]='3'

In [48]:
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

__Python VERSION: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]
__pyTorch VERSION: 1.13.1+cu117
__CUDA VERSION
/bin/bash: line 1: nvcc: command not found
__CUDNN VERSION: 8500
__Number CUDA Devices: 1
__Devices
index, name, driver_version, memory.total [MiB], memory.used [MiB], memory.free [MiB]
0, NVIDIA GeForce RTX 3090 Ti, 510.108.03, 24564 MiB, 20619 MiB, 3636 MiB
1, NVIDIA GeForce RTX 2080 Ti, 510.108.03, 11264 MiB, 6767 MiB, 4251 MiB
2, NVIDIA GeForce RTX 2080 Ti, 510.108.03, 11264 MiB, 8691 MiB, 2327 MiB
3, NVIDIA GeForce RTX 3090 Ti, 510.108.03, 24564 MiB, 8 MiB, 24247 MiB
4, NVIDIA GeForce RTX 3090 Ti, 510.108.03, 24564 MiB, 8 MiB, 24247 MiB
5, NVIDIA GeForce RTX 3090 Ti, 510.108.03, 24564 MiB, 8 MiB, 24247 MiB
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0


In [50]:
if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')

GPU


In [51]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig

In [52]:
from transformers import TrainingArguments, Trainer

model_checkpoint = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [53]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx <= len(self.x['input_ids']), (idx, len(self.x['input_ids']))
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n

In [54]:
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple
from sklearn.model_selection import train_test_split
import gc
from tqdm.auto import tqdm, trange

In [55]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [56]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [57]:
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0

    for batch in test_dataloader:
        with torch.no_grad():
            loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            num += len(batch) * loss.item()
            den += len(batch)
    val_loss = num / den
    return val_loss

In [58]:
import pandas as pd
from sklearn.utils import shuffle

In [59]:
path_data_en = "SemEval2018-Task9/training/data/1A.english.training.data.txt"
path_gold_en = "SemEval2018-Task9/training/gold/1A.english.training.gold.txt"

train_data_en = pd.read_csv(path_data_en, header=None, sep="\t", names=['term', 'relation'])
train_data_en.head()

,term,relation
0,blackfly,Concept
1,Turonian,Entity
2,abhorrence,Concept
3,tropical storm,Concept
4,militarization,Concept


In [60]:
train_data_en = 'hyponyms | ' + train_data_en.term.str.lower() +' | hypernyms:'
train_data_en.head()

0          hyponyms | blackfly | hypernyms:
1          hyponyms | turonian | hypernyms:
2        hyponyms | abhorrence | hypernyms:
3    hyponyms | tropical storm | hypernyms:
4    hyponyms | militarization | hypernyms:
Name: term, dtype: object

In [61]:
train_gold_en = pd.read_csv(path_gold_en, header=None, names=['hypernym'])
train_gold_en = train_gold_en.hypernym.str.split('\t').str.join(', ')
train_gold_en.head()

0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object

In [62]:
path_test_data_en = "SemEval2018-Task9/test/data/1A.english.test.data.txt"
path_test_gold_en = "SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

test_data_en = pd.read_csv(path_test_data_en, header=None, sep="\t", names=['term', 'relation'])
test_data_en = 'hyponyms | ' + test_data_en.term.str.lower() +' | hypernyms:'
test_data_en.head()

0    hyponyms | maliciousness | hypernyms:
1          hyponyms | buckler | hypernyms:
2        hyponyms | spelunker | hypernyms:
3     hyponyms | quo warranto | hypernyms:
4     hyponyms | jeff francis | hypernyms:
Name: term, dtype: object

In [63]:
test_gold_en = pd.read_csv(path_test_gold_en, header=None, names=['hypernym'])
test_gold_en = test_gold_en.hypernym.str.split('\t').str.join(', ')
test_gold_en.head()

0       malevolence, distaste, hatred, hate, malignity
1                                           body armor
2                    exploration, adventurer, explorer
3    proceedings, legal proceedings, proceeding, du...
4               thrower, baseball player, jock, person
Name: hypernym, dtype: object

In [64]:
from transformers import TrainingArguments, Trainer

model_checkpoint = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=100, block_size=64)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [65]:
train_dataset = PairsDataset(tokenizer(train_data_en.tolist()), tokenizer(train_gold_en.tolist()))
test_dataset = PairsDataset(tokenizer(test_data_en.tolist()), tokenizer(test_gold_en.tolist()))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

args = TrainingArguments(output_dir="t5-finetuned-large", num_train_epochs=15, per_device_train_batch_size=16, save_steps=10000)


In [66]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator
)

In [67]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1410
  Number of trainable parameters = 737668096
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.992800
1000,0.657400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1410, training_loss=0.7578492942431294, metrics={'train_runtime': 501.4459, 'train_samples_per_second': 44.87, 'train_steps_per_second': 2.812, 'total_flos': 1962716860416000.0, 'train_loss': 0.7578492942431294, 'epoch': 15.0})

In [68]:
trainer.save_model("t5-large_te_correct-no-train")

Saving model checkpoint to t5-large_te_correct-no-train
Configuration saved in t5-large_te_correct-no-train/config.json
Configuration saved in t5-large_te_correct-no-train/generation_config.json
Model weights saved in t5-large_te_correct-no-train/pytorch_model.bin
tokenizer config file saved in t5-large_te_correct-no-train/tokenizer_config.json
Special tokens file saved in t5-large_te_correct-no-train/special_tokens_map.json


In [69]:
model_saved = "./t5-large_te_correct-no-train"
model = T5ForConditionalGeneration.from_pretrained(model_saved).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_saved)

loading configuration file ./t5-large_te_correct-no-train/config.json
Model config T5Config {
  "_name_or_path": "t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
  

In [70]:
from collections import Counter

In [71]:
from tqdm import tqdm

In [97]:
test_pred_en=[]
for i_2 in tqdm(test_data_en.tolist()):
        input_ids = tokenizer.encode(i_2, return_tensors="pt")
        output_batch = model.generate(input_ids.cuda(), no_repeat_ngram_size=2, max_new_tokens=2048, 
                                      num_return_sequences=50, num_beams=50, early_stopping=True, 
                                      num_beam_groups=5, 
                                      diversity_penalty=1.0)
        decoded_list = []
        for outputs in output_batch:
            decoded = tokenizer.decode(outputs, skip_special_tokens=True).split(", ")
            decoded_list.extend(decoded)

        predicted_answer = set(decoded_list)
        sorted_predicted_answer = [i[0] for i in Counter(decoded_list).most_common()]
        

        _answers = set(sorted_predicted_answer)
        test_pred_en.append(_answers)
        

  0%|                                                                                          | 0/1500 [00:00<?, ?it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

  0%|                                                                                  | 1/1500 [00:00<09:00,  2.77it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

  0%|                                                                                  | 2/1500 [00:00<07:47,  3.20it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

  0%|▏                                                                                 | 3/1500 [00:00<08:09,  3.06it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id"

In [99]:
from random import shuffle

test_pred_en_df = []
for i in test_pred_en:
    test_pred_en_df.append('\t'.join(i))


test_pred_en_df = pd.DataFrame(test_pred_en_df)
test_pred_en_df.to_csv('test_pred_en.txt', header=None, index=None)


In [101]:
!python SemEval2018-Task9/task9-scorer.py SemEval2018-Task9/test/gold/1A.english.test.gold.txt test_pred_en.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
MRR: 0.08740101750101756
MAP: 0.07201211257877937
P@1: 0.026
P@3: 0.02522222222222222
P@5: 0.03351111111111121
P@15: 0.15450428460428423


In [ ]:
# del trainer
# cleanup()